# Sentiment evaluation

In [ ]:
from transformers import AutoModelForSequenceClassification
from transformers import TFAutoModelForSequenceClassification
from transformers import AutoTokenizer
import numpy as np
from scipy.special import softmax
import csv
import urllib.request
from tqdm.notebook import tqdm
from pathlib import Path
import pandas as pd
import swifter

In [ ]:
MAPPING = {
    0: "negative",
    1: "neutral",
    2: "positive"
}

FILENAME = "Ewaluacja ChatGPT - zadania - TweetEval - sentiment.csv"

In [ ]:
# Preprocess text (username and link placeholders)
def preprocess(text):
    new_text = []
    for t in text.split(" "):
        t = '@user' if t.startswith('@') and len(t) > 1 else t
        t = 'http' if t.startswith('http') else t
        new_text.append(t)
    return " ".join(new_text)

In [ ]:
def get_model(task: str = 'sentiment'):
    model_name = Path(f"cardiffnlp/twitter-roberta-base-{task}")
    tokenizer_name = model_name / "tokenizer"
    
    if not tokenizer_name.exists():
        tokenizer = AutoTokenizer.from_pretrained(model_name)
    else:
        tokenizer = AutoTokenizer.from_pretrained(tokenizer_name)
        
    model = AutoModelForSequenceClassification.from_pretrained(model_name)
    model.save_pretrained(model_name)
    tokenizer.save_pretrained(tokenizer_name)

    return model, tokenizer

In [ ]:
def preprocess_and_tokenize(row, tokenizer):
    row = preprocess(row)
    return tokenizer(row, return_tensors='pt')

## RoBERTa-retrained 

In [ ]:
labels=[]
mapping_link = f"https://raw.githubusercontent.com/cardiffnlp/tweeteval/main/datasets/sentiment/mapping.txt"
with urllib.request.urlopen(mapping_link) as f:
    html = f.read().decode('utf-8').split("\n")
    csvreader = csv.reader(html, delimiter='\t')
labels = [row[1] for row in csvreader if len(row) > 1]
model, tokenizer = get_model()

In [ ]:
df = pd.read_csv(FILENAME)
df

,id,annotation,text,promptid,prompt,gptid,chatgpt_final,chatgpt_raw,comment
0,0,1,@user @user what do these '1/2 naked pics' hav...,0,\nDescribe the sentiment of the given text. Ch...,0.0,0,0,NaN
1,1,1,OH: “I had a blue penis while I was this” [pla...,1,\nDescribe the sentiment of the given text. Ch...,1.0,0,0,NaN
2,2,1,"@user @user That's coming, but I think the vic...",2,\nDescribe the sentiment of the given text. Ch...,2.0,0,0,NaN
3,3,2,I think I may be finally in with the in crowd ...,3,\nDescribe the sentiment of the given text. Ch...,3.0,2,2,NaN
4,4,0,"@user Wow,first Hugo Chavez and now Fidel Cast...",4,\nDescribe the sentiment of the given text. Ch...,4.0,0,0 (negative),NaN
...,...,...,...,...,...,...,...,...,...
5138,10849,2,David Blaine Beyond Magic GREATEST TRICKS EXPO...,10849,\nDescribe the sentiment of the given text. Ch...,NaN,1,1,NaN
5139,10850,0,"And so privatisation takes root, and begins to...",10850,\nDescribe the sentiment of the given text. Ch...,NaN,0,0,NaN
5140,10851,1,Senior hurriyat leaders Ali Geelani Mirwaiz Um...,10851,\nDescribe the sentiment of the given text. Ch...,NaN,1,1,NaN
5141,10852,2,@user @user I'd prefer Ashley Graham if we're ...,10852,\nDescribe the sentiment of the given text. Ch...,NaN,1,1,NaN


In [ ]:
df["tokenized"] = df["text"].swifter.apply(lambda r: preprocess_and_tokenize(r, tokenizer))
df

Pandas Apply:   0%|          | 0/5143 [00:00<?, ?it/s]

,id,annotation,text,promptid,prompt,gptid,chatgpt_final,chatgpt_raw,comment,tokenized
0,0,1,@user @user what do these '1/2 naked pics' hav...,0,\nDescribe the sentiment of the given text. Ch...,0.0,0,0,NaN,"[input_ids, attention_mask]"
1,1,1,OH: “I had a blue penis while I was this” [pla...,1,\nDescribe the sentiment of the given text. Ch...,1.0,0,0,NaN,"[input_ids, attention_mask]"
2,2,1,"@user @user That's coming, but I think the vic...",2,\nDescribe the sentiment of the given text. Ch...,2.0,0,0,NaN,"[input_ids, attention_mask]"
3,3,2,I think I may be finally in with the in crowd ...,3,\nDescribe the sentiment of the given text. Ch...,3.0,2,2,NaN,"[input_ids, attention_mask]"
4,4,0,"@user Wow,first Hugo Chavez and now Fidel Cast...",4,\nDescribe the sentiment of the given text. Ch...,4.0,0,0 (negative),NaN,"[input_ids, attention_mask]"
...,...,...,...,...,...,...,...,...,...,...
5138,10849,2,David Blaine Beyond Magic GREATEST TRICKS EXPO...,10849,\nDescribe the sentiment of the given text. Ch...,NaN,1,1,NaN,"[input_ids, attention_mask]"
5139,10850,0,"And so privatisation takes root, and begins to...",10850,\nDescribe the sentiment of the given text. Ch...,NaN,0,0,NaN,"[input_ids, attention_mask]"
5140,10851,1,Senior hurriyat leaders Ali Geelani Mirwaiz Um...,10851,\nDescribe the sentiment of the given text. Ch...,NaN,1,1,NaN,"[input_ids, attention_mask]"
5141,10852,2,@user @user I'd prefer Ashley Graham if we're ...,10852,\nDescribe the sentiment of the given text. Ch...,NaN,1,1,NaN,"[input_ids, attention_mask]"


In [ ]:
bert_results = {
    "text": [],
    "roberta": [],
    "confidence": [],
    "annotation": [],
}

for id_, row in tqdm(df.iterrows()):
    encoded_input = row["tokenized"]
    output = model(**encoded_input)
    scores = output[0][0].detach().numpy()
    scores = softmax(scores)

    ranking = np.argsort(scores)
    ranking = ranking[::-1]

    bert_results["roberta"].append(labels[ranking[0]])
    bert_results["confidence"].append(scores[ranking[0]])
    bert_results["text"].append(row["text"])

# MULTILINGUAL MODEL
    bert_results["annotation"].append(row["annotation"])

0it [00:00, ?it/s]

In [ ]:
results = pd.DataFrame(bert_results)
results["roberta_final"] = results["roberta"].swifter.apply(lambda val: 0 if val == "negative" else 1 if val == "neutral" else 2)

Pandas Apply:   0%|          | 0/5143 [00:00<?, ?it/s]

In [ ]:
results.to_csv("RobertaBase-sentiment.csv", index=False)

## TimeLMS

In [ ]:
!pip install tweetnlp

In [ ]:
import tweetnlp

In [ ]:
model = tweetnlp.load_model('sentiment')  # Or `model = tweetnlp.Sentiment()` 

timelms_results = {
    "text": [],
    "roberta": [],
    "confidence": [],
    "annotation": [],
}

for id_, row in tqdm(df.iterrows()):
    output = model.sentiment(row["text"], return_probability=True)
    
    timelms_results["roberta"].append(output["label"])
    timelms_results["confidence"].append(output['probability'][output["label"]])
    timelms_results["text"].append(row["text"])
    timelms_results["annotation"].append(row["annotation"])

Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


0it [00:00, ?it/s]

In [ ]:
results = pd.DataFrame(timelms_results)
results["roberta_final"] = results["roberta"].swifter.apply(lambda val: 0 if val == "negative" else 1 if val == "neutral" else 2)

Pandas Apply:   0%|          | 0/5143 [00:00<?, ?it/s]

In [ ]:
results

,text,roberta,confidence,annotation,roberta_final
0,@user @user what do these '1/2 naked pics' hav...,negative,0.804726,1,0
1,OH: “I had a blue penis while I was this” [pla...,neutral,0.866949,1,1
2,"@user @user That's coming, but I think the vic...",neutral,0.763724,1,1
3,I think I may be finally in with the in crowd ...,positive,0.774047,2,2
4,"@user Wow,first Hugo Chavez and now Fidel Cast...",neutral,0.416397,0,1
...,...,...,...,...,...
5138,David Blaine Beyond Magic GREATEST TRICKS EXPO...,positive,0.876698,2,2
5139,"And so privatisation takes root, and begins to...",negative,0.941841,0,0
5140,Senior hurriyat leaders Ali Geelani Mirwaiz Um...,neutral,0.916536,1,1
5141,@user @user I'd prefer Ashley Graham if we're ...,neutral,0.735311,2,1


In [ ]:
results.to_csv("Timelms-sentiment.csv", index=False)